In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim  

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

from textblob import TextBlob

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

C:\Users\HP\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\Users\HP\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


In [2]:
df1 = pd.read_csv('ethiopia_tripadvisor_data_2.csv')
df1.head()

,Destination Name,Destination Type,Rating,Polarity,Date,Month,Year,Year Quarter,Review Head,Review Body,Review Text,Review Head Length,Review Body Length,Review Text Length,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text
0,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,72,179,252,-0.05,0.300000,0.125000
1,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,34,233,268,0.25,-0.150000,-0.016667
2,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopi...,22,103,126,0.90,0.345000,0.483750
3,Rock-Hewn Churches of Lalibela,Landmarks,4,1,October 2019,October,2019,4,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,7,241,249,0.60,0.225000,0.350000
4,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,15,243,259,0.20,0.233333,0.228571


In [3]:
df1['Destination Type'].value_counts()

Landmarks       3803
Museums         2507
Nature/Parks    2001
Name: Destination Type, dtype: int64

In [4]:
df1['Destination Name'].value_counts()

Rock-Hewn Churches of Lalibela                   1178
National Museum of Ethiopia                      1155
Simien Mountains National Park                   1073
Red Terror Martyrs Memorial Museum                668
Omo National Park and River                       454
Holy Trinity Cathedral                            442
Ethnological Museum                               397
Debre Berhan Selassie Church                      227
Fasiladas Bath Timket                             205
Church of St. George                              185
The Ruins of Aksum                                166
St. George's Cathedral                            162
Yemrehanna Kristos                                155
Bale Mountains                                    127
Entoto Maryam Church                              125
Abuna Yemata Guh church                           112
Nechisar National Park                             95
Harar Jegol Wall                                   83
Gheralta Rock               

In [3]:
df2 = df1.copy()
df2.head()

,Destination Name,Destination Type,Rating,Polarity,Date,Month,Year,Year Quarter,Review Head,Review Body,Review Text,Review Head Length,Review Body Length,Review Text Length,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text
0,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,72,179,252,-0.05,0.300000,0.125000
1,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,34,233,268,0.25,-0.150000,-0.016667
2,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopi...,22,103,126,0.90,0.345000,0.483750
3,Rock-Hewn Churches of Lalibela,Landmarks,4,1,October 2019,October,2019,4,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,7,241,249,0.60,0.225000,0.350000
4,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,15,243,259,0.20,0.233333,0.228571


In [4]:
# Remove new line characters
df2['Review Text'] = df2['Review Text'].apply(lambda x: re.sub('\s+', ' ', x))
df2['Review Head'] = df2['Review Head'].apply(lambda x: re.sub('\s+', ' ', x))
df2['Review Body'] = df2['Review Body'].apply(lambda x: re.sub('\s+', ' ', x))
df2.head()

,Destination Name,Destination Type,Rating,Polarity,Date,Month,Year,Year Quarter,Review Head,Review Body,Review Text,Review Head Length,Review Body Length,Review Text Length,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text
0,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,72,179,252,-0.05,0.300000,0.125000
1,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,34,233,268,0.25,-0.150000,-0.016667
2,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,22,103,126,0.90,0.345000,0.483750
3,Rock-Hewn Churches of Lalibela,Landmarks,4,1,October 2019,October,2019,4,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,7,241,249,0.60,0.225000,0.350000
4,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,15,243,259,0.20,0.233333,0.228571


In [5]:
df2['Review Text'].values

array(["Incredible locations don't be put off thinking these are boring churches Amazing location must visit. This site is marketed as churches but I think it is actually a rock city as there are no records of when they were built and who built them. Must see.",
       'It is a real miracle of the world! I do recommend to visit Lalibela churches and then have a mule trip to the mountain monastery. I would say you need at least two days for this city. I can also recommend a local guide who can arrange tours to all over in Ethiopia...',
       'Incredible experience A must visit in Ethiopia - one of the most unique experiences I have had in Africa. Highly recommended.',
       ...,
       'We spend eight glorious days in the park with or Guide Samual Kasaw and his crew of chefs and baggage handlers We ( chefs from the world) Heinz Egli ( Hong Kong) Urs Besmer ( Hong Kong) Fritz Gross ( Hong Kong) Romano Venuti ( Philippines) Ruddy Muller ( Disney HK) Martin Kobald ( Vice president WACS &

In [6]:
df2['TextBlob Polarity Head'] = df2['Review Head'].map(lambda text: TextBlob(text).sentiment.polarity)
df2['TextBlob Polarity Body'] = df2['Review Body'].map(lambda text: TextBlob(text).sentiment.polarity)
df2['TextBlob Polarity Text'] = df2['Review Text'].map(lambda text: TextBlob(text).sentiment.polarity)
df2.head()

,Destination Name,Destination Type,Rating,Polarity,Date,Month,Year,Year Quarter,Review Head,Review Body,Review Text,Review Head Length,Review Body Length,Review Text Length,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text
0,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,72,179,252,-0.05,0.300000,0.125000
1,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,34,233,268,0.25,-0.150000,-0.016667
2,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,22,103,126,0.90,0.345000,0.483750
3,Rock-Hewn Churches of Lalibela,Landmarks,4,1,October 2019,October,2019,4,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,7,241,249,0.60,0.225000,0.350000
4,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,15,243,259,0.20,0.233333,0.228571


In [7]:
df2[df2['Rating'] < 3]['Review Body'].values

array(["I can't agree with everyone more in that these churches are spectacular. everyone should see the. Today however I witnessed very bad behavior by a guide, July is off season and there are not many tourists here - this is great because you can see...",
       'The entry ticket of 50$ - which go directly in the pockets of the holy Ethiopian clergy - is a joke. The place is dirty, full of trash and populated by annoying touts. When all these people begging for money would aim their effort at...',
       'I understand to pay entrance and to Support the community... But 10 years ago it was 1 usd entrance and now 50 usd! I guess the priests figured out how to make business with their religion... I would have paid 10-15 usd but 50 seems...',
       'Sometime after Lalibela was declared a World Heritage site UNESCO sponsored a series of absolutely ghastly steel and concrete structures to cover all of the temples (except St George) making them impossible to view, enjoy and photograph fro

### SpaCy + Gensim

In [9]:
nlp = spacy.load('en_core_web_sm')

ethiopia_stop_words = ['ethiopia', 'ethiopian', 'rock', 'hewn', 'church', 'churches', 'park', 'parks', 'museum', 
                       'museums', 'omo', 'valley', 'mountain', 'mountains', 'addis', 'ababa', 'lucy', 'simien', 
                       'national', 'lalibela', 'gelada', 'haile', 'selassie', 'gondar', 'africa' , 'harar', 
                       'cathedral', 'cathedrals', 'place', 'visit', 'girmay', 
                       '\n', '‘s']

nlp.Defaults.stop_words.update(ethiopia_stop_words)

for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [10]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)

def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

nlp.add_pipe(lemmatizer,name = 'lemmatizer',after = 'ner')
nlp.add_pipe(remove_stopwords, name = 'stopwords', last = True)

In [11]:
# Adjust wanted corpus for clustering

# newest_doc = df2['Review Text']
# newest_doc = df2[df2['Polarity'] == 0]['Review Text']
# newest_doc = df2[df2['Polarity'] == 1]['Review Text']
# newest_doc = df2[df2['Rating'] == 1]['Review Text']
# newest_doc = df2[df2['Rating'] == 2]['Review Text']
# newest_doc = df2[df2['Rating'] == 3]['Review Text']
# newest_doc = df2[df2['Rating'] == 4]['Review Text']
# newest_doc = df2[df2['Rating'] == 5]['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks']['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks'][df2['TextBlob Polarity Text'] >= 0]['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks'][df2['TextBlob Polarity Text'] < 0]['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks']['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks'][df2['TextBlob Polarity Text'] >= 0]['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks'][df2['TextBlob Polarity Text'] < 0]['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Museums']['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Museums'][df2['TextBlob Polarity Text'] >= 0]['Review Text']
# newest_doc = df2[df2['Destination Type'] == 'Museums'][df2['TextBlob Polarity Text'] < 0]['Review Text']
# newest_doc = df2[df2['TextBlob Polarity Text'] >= 0]['Review Text']
# newest_doc = df2[df2['TextBlob Polarity Text'] < 0]['Review Text']

# newest_doc = df2['Review Head']
newest_doc = df2[df2['Polarity'] == 0]['Review Head']
# newest_doc = df2[df2['Polarity'] == 1]['Review Head']
# newest_doc = df2[df2['Rating'] == 1]['Review Head']
# newest_doc = df2[df2['Rating'] == 2]['Review Head']
# newest_doc = df2[df2['Rating'] == 3]['Review Head']
# newest_doc = df2[df2['Rating'] == 4]['Review Head']
# newest_doc = df2[df2['Rating'] == 5]['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks']['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks'][df2['TextBlob Polarity Text'] >= 0]['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks'][df2['TextBlob Polarity Text'] < 0]['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks']['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks'][df2['TextBlob Polarity Text'] >= 0]['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks'][df2['TextBlob Polarity Text'] < 0]['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Museums']['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Museums'][df2['TextBlob Polarity Text'] >= 0]['Review Head']
# newest_doc = df2[df2['Destination Type'] == 'Museums'][df2['TextBlob Polarity Text'] < 0]['Review Head']
# newest_doc = df2[df2['TextBlob Polarity Head'] >= 0]['Review Head']
# newest_doc = df2[df2['TextBlob Polarity Head'] < 0]['Review Head']

# newest_doc = df2['Review Body']
# newest_doc = df2[df2['Polarity'] == 0]['Review Body']
# newest_doc = df2[df2['Polarity'] == 1]['Review Body']
# newest_doc = df2[df2['Rating'] == 1]['Review Body']
# newest_doc = df2[df2['Rating'] == 2]['Review Body']
# newest_doc = df2[df2['Rating'] == 3]['Review Body']
# newest_doc = df2[df2['Rating'] == 4]['Review Body']
# newest_doc = df2[df2['Rating'] == 5]['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks']['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks'][df2['TextBlob Polarity Text'] >= 0]['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Landmarks'][df2['TextBlob Polarity Text'] < 0]['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks']['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks'][df2['TextBlob Polarity Text'] >= 0]['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Nature/Parks'][df2['TextBlob Polarity Text'] < 0]['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Museums']['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Museums'][df2['TextBlob Polarity Text'] >= 0]['Review Body']
# newest_doc = df2[df2['Destination Type'] == 'Museums'][df2['TextBlob Polarity Text'] < 0]['Review Body']
# newest_doc = df2[df2['TextBlob Polarity Body'] >= 0]['Review Body']
# newest_doc = df2[df2['TextBlob Polarity Body'] < 0]['Review Body']

In [12]:
doc_list = []
# Iterates through each article in the corpus.
for doc in tqdm(newest_doc):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append(pr)

# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = words,
                                           num_topics = 3, 
                                           random_state = 2,
                                           update_every = 1,
                                           passes = 10,
                                           alpha = 'auto',
                                           per_word_topics = True)

pprint(lda_model.print_topics(num_words = 10))

[(0,
  '0.021*"need" + 0.018*"price" + 0.015*"poorly" + 0.013*"little" + '
  '0.013*"rip" + 0.010*"guide" + 0.010*"view" + 0.010*"experience" + '
  '0.010*"history" + 0.007*"5"'),
 (1,
  '0.038*"worth" + 0.029*"guide" + 0.029*"disappointing" + 0.013*"money" + '
  '0.013*"time" + 0.011*"expensive" + 0.011*"bad" + 0.011*"small" + '
  '0.011*"poor" + 0.009*"beautiful"'),
 (2,
  '0.026*"tour" + 0.013*"underwhelme" + 0.013*"good" + 0.010*"way" + '
  '0.010*"avoid" + 0.010*"Guide" + 0.010*"let" + 0.010*"experience" + '
  '0.010*"nice" + 0.007*"bit"')]


### Scratch

In [23]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words1 = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
stop_words2 = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]

In [25]:
stop_words3 = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "A", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "after", "afterwards", "ag", "again", "against", "ah", "ain", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appreciate", "approximately", "ar", "are", "aren", "arent", "arise", "around", "as", "aside", "ask", "asking", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "B", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "been", "before", "beforehand", "beginnings", "behind", "below", "beside", "besides", "best", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "C", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "ci", "cit", "cj", "cl", "clearly", "cm", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "could", "couldn", "couldnt", "course", "cp", "cq", "cr", "cry", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d", "D", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "dj", "dk", "dl", "do", "does", "doesn", "doing", "don", "done", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "E", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "F", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "G", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "H", "h2", "h3", "had", "hadn", "happens", "hardly", "has", "hasn", "hasnt", "have", "haven", "having", "he", "hed", "hello", "help", "hence", "here", "hereafter", "hereby", "herein", "heres", "hereupon", "hes", "hh", "hi", "hid", "hither", "hj", "ho", "hopefully", "how", "howbeit", "however", "hr", "hs", "http", "hu", "hundred", "hy", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "im", "immediately", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "inward", "io", "ip", "iq", "ir", "is", "isn", "it", "itd", "its", "iv", "ix", "iy", "iz", "j", "J", "jj", "jr", "js", "jt", "ju", "just", "k", "K", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "ko", "l", "L", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "M", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "my", "n", "N", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "neither", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "O", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "otherwise", "ou", "ought", "our", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "P", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "pp", "pq", "pr", "predominantly", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "Q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "R", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "S", "s2", "sa", "said", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "seem", "seemed", "seeming", "seems", "seen", "sent", "seven", "several", "sf", "shall", "shan", "shed", "shes", "show", "showed", "shown", "showns", "shows", "si", "side", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somehow", "somethan", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "sz", "t", "T", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "thereof", "therere", "theres", "thereto", "thereupon", "these", "they", "theyd", "theyre", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "U", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "used", "useful", "usefully", "usefulness", "using", "usually", "ut", "v", "V", "va", "various", "vd", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "W", "wa", "was", "wasn", "wasnt", "way", "we", "wed", "welcome", "well", "well-b", "went", "were", "weren", "werent", "what", "whatever", "whats", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "whom", "whomever", "whos", "whose", "why", "wi", "widely", "with", "within", "without", "wo", "won", "wonder", "wont", "would", "wouldn", "wouldnt", "www", "x", "X", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "Y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "your", "youre", "yours", "yr", "ys", "yt", "z", "Z", "zero", "zi", "zz"]

In [26]:
stop_words4 = list(set(set(stop_words1) | set(stop_words2) | set(stop_words3)))

In [27]:
ethiopia_stop_words = ['ethiopia', 'rock', 'hewn', 'church', 'churches', 'park', 'parks', 'museum', 'museums', 'omo', 'valley',
                       'mountain', 'mountains', 'addis', 'ababa']

In [28]:
stop_words5 = list(set(set(stop_words4) | set(ethiopia_stop_words)))